In [1]:
#from generator import UNETGenerator
from keras.layers import Activation, Input, Dropout, merge,concatenate
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers import Conv2D,Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model

Using TensorFlow backend.


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# ---------------------------------------------
# HYPER PARAMS
# ---------------------------------------------
# width, height of images to work with. Assumes images are square
im_width = im_height = 256

# inpu/oputputt channels in image
input_channels = 3
output_channels = 3

# image dims
input_img_dim = (im_width, im_height,input_channels)
output_img_dim = (im_width, im_height,output_channels)

In [4]:
def UNETGenerator(input_img_dim, num_output_channels):
    """
    Creates the generator according to the specs in the paper below.
    It's basically a skip layer AutoEncoder
    Generator does the following:
    1. Takes in an image
    2. Generates an image from this image
    Differs from a standard GAN because the image isn't random.
    This model tries to learn a mapping from a suboptimal image to an optimal image.
    [https://arxiv.org/pdf/1611.07004v1.pdf][5. Appendix]
    :param input_img_dim: (channel, height, width)
    :param output_img_dim: (channel, height, width)
    :return:
    """
    # -------------------------------
    # ENCODER
    # C64-C128-C256-C512-C512-C512-C512-C512
    # 1 layer block = Conv - BN - LeakyRelu
    # -------------------------------
    stride = 2
    merge_mode = 'concat'

    # batch norm mode
    bn_mode = 2

    # batch norm merge axis
    bn_axis = 3
    bn_axis_1 = 3

    input_layer = Input(shape=input_img_dim, name="unet_input")

    # 1 encoder C64
    # skip batchnorm on this layer on purpose (from paper)
    en_1 = Convolution2D(nb_filter=64, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(input_layer)
    #en_1 = Conv2D(filters=64,kernel_size=4,padding="same",strides=2)(input_layer)
    en_1 = LeakyReLU(alpha=0.2)(en_1)

#     # 2 encoder C128
    en_2 = Convolution2D(nb_filter=64, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_1)
    #en_2 = Conv2D(filters=64,kernel_size=4,padding="same",strides=2)(en_1)
    en_2 = BatchNormalization(name='gen_en_bn_2', axis=bn_axis)(en_2)
    en_2 = LeakyReLU(alpha=0.2)(en_2)

    # 3 encoder C256
    en_3 = Convolution2D(nb_filter=256, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_2)
    en_3 = BatchNormalization(name='gen_en_bn_3', axis=bn_axis)(en_3)
    en_3 = LeakyReLU(alpha=0.2)(en_3)

    # 4 encoder C512
    en_4 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_3)
    en_4 = BatchNormalization(name='gen_en_bn_4', axis=bn_axis)(en_4)
    en_4 = LeakyReLU(alpha=0.2)(en_4)

    # 5 encoder C512
    en_5 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_4)
    en_5 = BatchNormalization(name='gen_en_bn_5', axis=bn_axis)(en_5)
    en_5 = LeakyReLU(alpha=0.2)(en_5)

    # 6 encoder C512
    en_6 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_5)
    en_6 = BatchNormalization(name='gen_en_bn_6', axis=bn_axis)(en_6)
    en_6 = LeakyReLU(alpha=0.2)(en_6)

    # 7 encoder C512
    en_7 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_6)
    en_7 = BatchNormalization(name='gen_en_bn_7', axis=bn_axis)(en_7)
    en_7 = LeakyReLU(alpha=0.2)(en_7)

    # 8 encoder C512
    en_8 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same', subsample=(stride, stride))(en_7)
    en_8 = BatchNormalization(name='gen_en_bn_8', axis=bn_axis)(en_8)
    en_8 = LeakyReLU(alpha=0.2)(en_8)
    
#    unet_generator = Model(input=[input_layer], output=[en_8], name='unet_generator')


    # -------------------------------
    # DECODER
    # CD512-CD1024-CD1024-C1024-C1024-C512-C256-C128
    # 1 layer block = Conv - Upsample - BN - DO - Relu
    # also adds skip connections (merge). Takes input from previous layer matching encoder layer
    # -------------------------------
    # 1 decoder CD512 (decodes en_8)
    de_1 = UpSampling2D(size=(2, 2))(en_8)
    de_1 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same')(de_1)
    de_1 = BatchNormalization(name='gen_de_bn_1', axis=bn_axis)(de_1)
    de_1 = Dropout(p=0.5)(de_1)
    #de_1 = merge([de_1, en_7], mode=merge_mode, concat_axis=1)
    de_1 = concatenate([de_1, en_7],axis = bn_axis_1)
    de_1 = Activation('relu')(de_1)

    # 2 decoder CD1024 (decodes en_7)
    de_2 = UpSampling2D(size=(2, 2))(de_1)
    de_2 = Convolution2D(nb_filter=1024, nb_row=4, nb_col=4, border_mode='same')(de_2)
    de_2 = BatchNormalization(name='gen_de_bn_2', axis=bn_axis)(de_2)
    de_2 = Dropout(p=0.5)(de_2)
    #de_2 = merge([de_2, en_6], mode=merge_mode, concat_axis=1)
    de_2 = concatenate([de_2, en_6],axis = bn_axis_1)
    de_2 = Activation('relu')(de_2)

    # 3 decoder CD1024 (decodes en_6)
    de_3 = UpSampling2D(size=(2, 2))(de_2)
    de_3 = Convolution2D(nb_filter=1024, nb_row=4, nb_col=4, border_mode='same')(de_3)
    de_3 = BatchNormalization(name='gen_de_bn_3', axis=bn_axis)(de_3)
    de_3 = Dropout(p=0.5)(de_3)
    #de_3 = merge([de_3, en_5], mode=merge_mode, concat_axis=1)
    de_3 = concatenate([de_3, en_5],axis = bn_axis_1)
    de_3 = Activation('relu')(de_3)

    # 4 decoder CD1024 (decodes en_5)
    de_4 = UpSampling2D(size=(2, 2))(de_3)
    de_4 = Convolution2D(nb_filter=1024, nb_row=4, nb_col=4, border_mode='same')(de_4)
    de_4 = BatchNormalization(name='gen_de_bn_4', axis=bn_axis)(de_4)
    de_4 = Dropout(p=0.5)(de_4)
    #de_4 = merge([de_4, en_4], mode=merge_mode, concat_axis=1)
    de_4 = concatenate([de_4, en_4],axis = bn_axis_1)
    de_4 = Activation('relu')(de_4)

    # 5 decoder CD1024 (decodes en_4)
    de_5 = UpSampling2D(size=(2, 2))(de_4)
    de_5 = Convolution2D(nb_filter=1024, nb_row=4, nb_col=4, border_mode='same')(de_5)
    de_5 = BatchNormalization(name='gen_de_bn_5', axis=bn_axis)(de_5)
    de_5 = Dropout(p=0.5)(de_5)
    #de_5 = merge([de_5, en_3], mode=merge_mode, concat_axis=1)
    de_5 = concatenate([de_5, en_3],axis = bn_axis_1)
    de_5 = Activation('relu')(de_5)

    # 6 decoder C512 (decodes en_3)
    de_6 = UpSampling2D(size=(2, 2))(de_5)
    de_6 = Convolution2D(nb_filter=512, nb_row=4, nb_col=4, border_mode='same')(de_6)
    de_6 = BatchNormalization(name='gen_de_bn_6', axis=bn_axis)(de_6)
    de_6 = Dropout(p=0.5)(de_6)
    #de_6 = merge([de_6, en_2], mode=merge_mode, concat_axis=1)
    de_6 = concatenate([de_6, en_2],axis = bn_axis_1)
    de_6 = Activation('relu')(de_6)

    # 7 decoder CD256 (decodes en_2)
    de_7 = UpSampling2D(size=(2, 2))(de_6)
    de_7 = Convolution2D(nb_filter=256, nb_row=4, nb_col=4, border_mode='same')(de_7)
    de_7 = BatchNormalization(name='gen_de_bn_7', axis=bn_axis)(de_7)
    de_7 = Dropout(p=0.5)(de_7)
    #de_7 = merge([de_7, en_1], mode=merge_mode, concat_axis=1)
    de_7 = concatenate([de_7, en_1],axis = bn_axis_1)
    de_7 = Activation('relu')(de_7)

    # After the last layer in the decoder, a convolution is applied
    # to map to the number of output channels (3 in general,
    # except in colorization, where it is 2), followed by a Tanh
    # function.
    de_8 = UpSampling2D(size=(2, 2))(de_7)
    de_8 = Convolution2D(nb_filter=num_output_channels, nb_row=4, nb_col=4, border_mode='same')(de_8)
    de_8 = Activation('tanh')(de_8)

    unet_generator = Model(input=[input_layer], output=[de_8], name='unet_generator')
    return unet_generator

In [5]:
generator_nn = UNETGenerator(input_img_dim=input_img_dim, num_output_channels=output_channels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=64, strides=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=64, strides=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=256, strides=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=512, strides=(2, 2), padding="same")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(kernel_size=(4, 4), filters=512, strides=(2, 2), padding="same")`
/usr/local/l

In [8]:
generator_nn.summary()

Model: "unet_generator"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
unet_input (InputLayer)         (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 128, 128, 64) 3136        unet_input[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 128, 128, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 64, 64, 64)   65600       leaky_re_lu_1[0][0]              
_____________________________________________________________________________________

In [10]:
generator_nn.load_weights("model_39.h5")

In [13]:
!ls facades/train/1.jpg

facades/train/1.jpg


In [39]:
import cv2
my_pic = cv2.imread("./facades/train/1.jpg")
my_pic=my_pic[:,:256,:]
my_pic.shape

(256, 256, 3)

In [40]:
my_pic = my_pic.reshape(1,256,256,3)
my_pic = (my_pic - 127.5) / 127.5

In [41]:
fake_pic = generator_nn.predict(my_pic)

In [42]:
fake_pic

array([[[[ 1.        , -1.        , -1.        ],
         [ 1.        , -1.        , -1.        ],
         [ 1.        , -1.        , -1.        ],
         ...,
         [ 0.70464516, -0.9799732 , -0.99422675],
         [ 0.6355746 , -0.9565906 , -0.98624563],
         [ 0.6164619 , -0.91233754, -0.9566242 ]],

        [[ 1.        , -1.        , -1.        ],
         [ 1.        , -1.        , -1.        ],
         [ 1.        , -1.        , -1.        ],
         ...,
         [ 0.7876942 , -0.99162674, -0.99862915],
         [ 0.737912  , -0.9796591 , -0.9964038 ],
         [ 0.74015033, -0.95492494, -0.98687565]],

        [[ 1.        , -1.        , -1.        ],
         [ 1.        , -1.        , -1.        ],
         [ 1.        , -1.        , -1.        ],
         ...,
         [ 0.78322244, -0.9908547 , -0.998682  ],
         [ 0.74265695, -0.98034036, -0.9968295 ],
         [ 0.75228786, -0.9608872 , -0.98934287]],

        ...,

        [[ 0.4936157 , -0.76849294, -0

In [43]:
fake_pic = fake_pic * 127.5 + 127.5

In [44]:
cv2.imwrite("result.jpg",fake_pic[0])

True